In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

features = "dataset2_features.csv"
features = pd.read_csv(features, sep=',', header=0, index_col=None)
Y = features[['labels']]
# feat_not_theta = [i for i in features.columns if "Theta" not in i]
# X = features.drop(feat_not_theta, axis=1)
# not_time_feat = [i for i in features.columns if ("ERN" not in i) & ("Pe" not in i) & ("amp" not in i)]
# X = features.drop(not_time_feat, axis=1)

#not_freq_feat = [i for i in features.columns if ("Delta" not in i) & ("Theta" not in i) & ("Alpha" not in i) &
 #                ("Freq" not in i)]
#X = features.drop(not_freq_feat, axis=1)
# X = features.drop('Participant', axis=1)
X = features.drop('labels', axis=1)
print(len(X.columns)) 


1629


In [2]:
from mrmr import mrmr_classif
selected_features = mrmr_classif(X=X, y=Y, K=40)

100%|██████████| 40/40 [00:28<00:00,  1.40it/s]


In [3]:
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X[i]], axis=1)

In [4]:
f1score = []
precision = []
recall = []
specificity = []
npv = []
bal_acc = []

splits = 100
sss = StratifiedShuffleSplit(n_splits=splits, test_size=0.3, random_state=0)
svm = SVC(class_weight='balanced')
scaler = preprocessing.StandardScaler()
X_col = combined_df.columns
Y=np.array(Y)

for train_index, test_index in sss.split(combined_df, Y):
    X_train, X_test = combined_df.iloc[train_index, :], combined_df.iloc[test_index, :]
    Y_train, Y_test = Y[train_index], Y[test_index]
    scal = scaler.fit(X_train)
    X_train = scal.transform(X_train)  # Variables standardization
    X_test = scal.transform(X_test)  # Variables standardization
    X_train = pd.DataFrame(X_train, columns=X_col)
    X_test = pd.DataFrame(X_test, columns=X_col)
    clf = svm.fit(X_train, Y_train)
    Y_predicted = clf.predict(X_test)
    f1score.append(f1_score(Y_test, Y_predicted))
    precision.append(precision_score(Y_test, Y_predicted))  # Precision = Positive predictive value
    npv.append(precision_score(Y_test, Y_predicted, pos_label=0))  # Negative predictive value
    recall.append(recall_score(Y_test, Y_predicted))  # Recall = Sensitivity
    specificity.append(recall_score(Y_test, Y_predicted, pos_label=0))
    bal_acc.append(balanced_accuracy_score(Y_test, Y_predicted))

# print(np.mean(f1score))
# print(np.mean(precision))
print(np.mean(recall))
print(np.std(recall))
print(np.mean(specificity))
print(np.std(specificity))
# print(np.mean(npv))
print(np.mean(bal_acc))
print(np.std(bal_acc))

0.5771232876712328
0.026507291773589414
0.7865192982456141
0.012696245776507414
0.6818212929584234
0.01276152874444839
